### Arhum Zafar
<br>
<br>
A brief attempt of using reinforcement learning strategies to make a quick trading agent.

In [ ]:
%watermark

In [ ]:
#!pip install tensorflow-gpu==2.0.0.alpha0|

### Dependencies

In [8]:
import math
import random
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas_datareader as data_reader

from tqdm import tqdm_notebook, tqdm
from collections import deque

In [9]:
tf.__version__

'2.2.0'

#### Building the trading network

In [10]:
class AI_Trader():
  
  def __init__(self, state_size, action_space=3, model_name="AITrader"): #Stay, Buy, Sell
    
    self.state_size = state_size
    self.action_space = action_space
    self.memory = deque(maxlen=2000)
    self.inventory = []
    self.model_name = model_name
    
    self.gamma = 0.95
    self.epsilon = 1.0
    self.epsilon_final = 0.01
    self.epsilon_decay = 0.995
    
    self.model = self.model_builder()
    
  def model_builder(self):
    
    model = tf.keras.models.Sequential()
    
    model.add(tf.keras.layers.Dense(units=32, activation='relu', input_dim=self.state_size))
    
    model.add(tf.keras.layers.Dense(units=64, activation='relu'))
    
    model.add(tf.keras.layers.Dense(units=128, activation='relu'))
    
    model.add(tf.keras.layers.Dense(units=self.action_space, activation='linear'))
    
    model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(lr=0.001))
    
    return model
  
  def trade(self, state):
    
    if random.random() <= self.epsilon:
      return random.randrange(self.action_space)
    
    actions = self.model.predict(state)
    return np.argmax(actions[0])
  
  
  def batch_train(self, batch_size):
    
    batch = []
    for i in range(len(self.memory) - batch_size + 1, len(self.memory)):
      batch.append(self.memory[i])
      
    for state, action, reward, next_state, done in batch:
      reward = reward
      if not done:
        reward = reward + self.gamma * np.amax(self.model.predict(next_state)[0])
        
      target = self.model.predict(state)
      target[0][action] = reward
      
      self.model.fit(state, target, epochs=1, verbose=0)
      
    if self.epsilon > self.epsilon_final:
      self.epsilon *= self.epsilon_decay

#### Preprocessing

In [11]:
#Sigmoid
def sigmoid(x):
  return 1 / (1 + math.exp(-x))

#Price Format Function
def stocks_price_format(n):
  if n < 0:
    return "- $ {0:2f}".format(abs(n))
  else:
    return "$ {0:2f}".format(abs(n))

#### Load Dataset

In [12]:
def dataset_loader(stock_name):
  
  #Complete the dataset loader function
  dataset = data_reader.DataReader(stock_name, data_source="yahoo")
  
  start_date = str(dataset.index[0]).split()[0]
  end_date = str(dataset.index[-1]).split()[0]
  
  close = dataset['Close']

  return close

#### State Creator

In [13]:
def state_creator(data, timestep, window_size):
  
  starting_id = timestep - window_size + 1
  
  if starting_id >= 0:
    windowed_data = data[starting_id:timestep+1]
  else:
    windowed_data = - starting_id * [data[0]] + list(data[0:timestep+1])
    
  state = []
  for i in range(window_size - 1):
    state.append(sigmoid(windowed_data[i+1] - windowed_data[i]))
    
  return np.array([state])

#### Loading a dataset for a stock

In [14]:
stock_name = "AAPL"
data = dataset_loader(stock_name)

### Training

In [16]:
# hyperparameters

window_size = 10
episodes = 1000

batch_size = 32
data_samples = len(data) - 1

In [18]:
trader = AI_Trader(window_size)
trader.model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 32)                352       
_________________________________________________________________
dense_5 (Dense)              (None, 64)                2112      
_________________________________________________________________
dense_6 (Dense)              (None, 128)               8320      
_________________________________________________________________
dense_7 (Dense)              (None, 3)                 387       
Total params: 11,171
Trainable params: 11,171
Non-trainable params: 0
_________________________________________________________________


In [ ]:
for episode in range(1, episodes + 1):
  
  print("Episode: {}/{}".format(episode, episodes))
  
  state = state_creator(data, 0, window_size + 1)
  
  total_profit = 0
  trader.inventory = []
  
  for t in tqdm(range(data_samples)):
    
    action = trader.trade(state)
    
    next_state = state_creator(data, t+1, window_size + 1)
    reward = 0
    
    if action == 1: #Buying
      trader.inventory.append(data[t])
      print("AI Trader bought: ", stocks_price_format(data[t]))
      
    elif action == 2 and len(trader.inventory) > 0: #Selling
      buy_price = trader.inventory.pop(0)
      
      reward = max(data[t] - buy_price, 0)
      total_profit += data[t] - buy_price
      print("AI Trader sold: ", stocks_price_format(data[t]), " Profit: " + stocks_price_format(data[t] - buy_price) )
      
    if t == data_samples - 1:
      done = True
    else:
      done = False
      
    trader.memory.append((state, action, reward, next_state, done))
    
    state = next_state
    
    if done:
      print("########################")
      print("TOTAL PROFIT: {}".format(total_profit))
      print("########################")
    
    if len(trader.memory) > batch_size:
      trader.batch_train(batch_size)
      
  if episode % 10 == 0:
    trader.model.save("ai_trader_{}.h5".format(episode))
    

  0%|          | 0/1258 [00:00<?, ?it/s]

Episode: 1/1000
AI Trader bought:  $ 126.820000
AI Trader sold:  $ 128.509995  Profit: $ 1.689995
AI Trader bought:  $ 129.619995
AI Trader bought:  $ 125.220001
AI Trader sold:  $ 124.500000  Profit: - $ 5.119995
AI Trader bought:  $ 123.379997
AI Trader sold:  $ 122.989998  Profit: - $ 2.230003
AI Trader sold:  $ 122.370003  Profit: - $ 1.009995
AI Trader bought:  $ 115.129997
AI Trader sold:  $ 115.519997  Profit: $ 0.389999
AI Trader bought:  $ 119.720001
AI Trader sold:  $ 113.489998  Profit: - $ 6.230003
AI Trader bought:  $ 115.010002
AI Trader bought:  $ 105.760002
AI Trader sold:  $ 103.120003  Profit: - $ 11.889999
AI Trader bought:  $ 103.739998


  3%|▎         | 35/1258 [00:07<22:32,  1.11s/it]

AI Trader bought:  $ 112.760002


  3%|▎         | 37/1258 [00:11<32:31,  1.60s/it]

...

  8%|▊         | 95/1258 [03:43<51:27,  2.65s/it]

AI Trader bought:  $ 118.879997


  8%|▊         | 96/1258 [03:45<49:15,  2.54s/it]

AI Trader sold:  $ 118.029999  Profit: - $ 0.849998


  9%|▉         | 111/1258 [04:19<42:52,  2.24s/it]

Depending on whether you have a GPU or not, the loop time will vary. As the loop iterates, you will notice how the network makes stronger predictions over time; reinforcing its behavior over while iterating, yielding greater profit.